In [1]:
import pandas as pd

import bokeh.charts
import bokeh.charts.utils
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [31]:
df_store = pd.read_csv('store_data.csv', encoding = "ISO-8859-1", parse_dates=['Order Date'])
df_store.head()

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2015-01-07,1/8/15,4.5600,4,13.01,88522
1,20228,Not Specified,0.02,500.98,26.00,5,Ronnie Proctor,Delivery Truck,Home Office,Furniture,...,West,California,San Gabriel,91776,2015-06-13,6/15/15,4390.3665,12,6362.85,90193
2,21776,Critical,0.06,9.48,7.29,11,Marcus Dunlap,Regular Air,Home Office,Furniture,...,East,New Jersey,Roselle,7203,2015-02-15,2/17/15,-53.8096,22,211.15,90192
3,24844,Medium,0.09,78.69,19.99,14,Gwendolyn F Tyson,Regular Air,Small Business,Furniture,...,Central,Minnesota,Prior Lake,55372,2015-05-12,5/14/15,803.4705,16,1164.45,86838
4,24846,Medium,0.08,3.28,2.31,14,Gwendolyn F Tyson,Regular Air,Small Business,Office Supplies,...,Central,Minnesota,Prior Lake,55372,2015-05-12,5/13/15,-24.0300,7,22.23,86838


In [32]:
# Add columns for sales where profit is plus or zero to negative
df_sales = df_store
df_sales['Profitable'] = df_store['Sales'].where(df_store['Profit'] > 0, 0)
df_sales['Unprofitable'] = df_store['Sales'].where(df_store['Profit'] <= 0, 0)
df_sales.head()

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID,Profitable,Unprofitable
0,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Anacortes,98221,2015-01-07,1/8/15,4.5600,4,13.01,88522,13.01,0.00
1,20228,Not Specified,0.02,500.98,26.00,5,Ronnie Proctor,Delivery Truck,Home Office,Furniture,...,San Gabriel,91776,2015-06-13,6/15/15,4390.3665,12,6362.85,90193,6362.85,0.00
2,21776,Critical,0.06,9.48,7.29,11,Marcus Dunlap,Regular Air,Home Office,Furniture,...,Roselle,7203,2015-02-15,2/17/15,-53.8096,22,211.15,90192,0.00,211.15
3,24844,Medium,0.09,78.69,19.99,14,Gwendolyn F Tyson,Regular Air,Small Business,Furniture,...,Prior Lake,55372,2015-05-12,5/14/15,803.4705,16,1164.45,86838,1164.45,0.00
4,24846,Medium,0.08,3.28,2.31,14,Gwendolyn F Tyson,Regular Air,Small Business,Office Supplies,...,Prior Lake,55372,2015-05-12,5/13/15,-24.0300,7,22.23,86838,0.00,22.23


In [33]:
# Filter only columns needed
df_sales = df_sales[['Order Date', 'Customer Segment', 'Profit', 'Sales', 'Profitable', 'Unprofitable']]
df_sales.head()

,Order Date,Customer Segment,Profit,Sales,Profitable,Unprofitable
0,2015-01-07,Corporate,4.5600,13.01,13.01,0.00
1,2015-06-13,Home Office,4390.3665,6362.85,6362.85,0.00
2,2015-02-15,Home Office,-53.8096,211.15,0.00,211.15
3,2015-05-12,Small Business,803.4705,1164.45,1164.45,0.00
4,2015-05-12,Small Business,-24.0300,22.23,0.00,22.23


In [44]:
# Group by Segment
df_sales2 = df_sales.groupby(['Customer Segment', 'Order Date']).sum()
df_sales2 = df_sales2.reset_index(drop=False)
df_sales2.head()

,Customer Segment,Order Date,Profit,Sales,Profitable,Unprofitable
0,Consumer,2015-01-01,-61.19400,609.09,0.00,609.09
1,Consumer,2015-01-02,2628.05002,4265.84,4046.96,218.88
2,Consumer,2015-01-03,90.62460,131.34,131.34,0.00
3,Consumer,2015-01-05,346.56910,1286.47,680.00,606.47
4,Consumer,2015-01-06,-733.70280,1918.27,1302.36,615.91


In [45]:
# Group by Month
df_mo = df_sales2.set_index('Order Date').groupby('Customer Segment').resample('M').sum()
df_mo = df_mo.reset_index(drop=False)
df_mo.head()

,Customer Segment,Order Date,Profit,Sales,Profitable,Unprofitable
0,Consumer,2015-01-31,15638.107680,57373.47,35650.12,21723.35
1,Consumer,2015-02-28,5722.565556,64397.91,38294.70,26103.21
2,Consumer,2015-03-31,-12732.196640,56318.78,34993.59,21325.19
3,Consumer,2015-04-30,19586.214664,93307.01,77728.49,15578.52
4,Consumer,2015-05-31,8373.289823,31557.43,18848.72,12708.71


In [46]:
# Change column header to remove space for using as index
df_segs = df_mo.rename(columns={'Customer Segment': 'Segment'})
df_segs.head()

,Segment,Order Date,Profit,Sales,Profitable,Unprofitable
0,Consumer,2015-01-31,15638.107680,57373.47,35650.12,21723.35
1,Consumer,2015-02-28,5722.565556,64397.91,38294.70,26103.21
2,Consumer,2015-03-31,-12732.196640,56318.78,34993.59,21325.19
3,Consumer,2015-04-30,19586.214664,93307.01,77728.49,15578.52
4,Consumer,2015-05-31,8373.289823,31557.43,18848.72,12708.71


In [50]:
# Would prefer a better solution as 'Customer Segment' doesn't work
df_segs.Segment.unique()[0]

'Consumer'

In [51]:
# Setting unique axis to pick up single Customer Segment
cons_df = df_segs[df_segs['Segment']==df_segs.Segment.unique()[0]]
cons_df

,Segment,Order Date,Profit,Sales,Profitable,Unprofitable
0,Consumer,2015-01-31,15638.107680,57373.47,35650.12,21723.35
1,Consumer,2015-02-28,5722.565556,64397.91,38294.70,26103.21
2,Consumer,2015-03-31,-12732.196640,56318.78,34993.59,21325.19
3,Consumer,2015-04-30,19586.214664,93307.01,77728.49,15578.52
4,Consumer,2015-05-31,8373.289823,31557.43,18848.72,12708.71
5,Consumer,2015-06-30,13307.009190,98823.77,83627.65,15196.12


In [56]:
# Filter only columns for chart
cons_df_area = cons_df[['Order Date', 'Profitable', 'Unprofitable']]
cons_df_area.head()

,Order Date,Profitable,Unprofitable
0,2015-01-31,35650.12,21723.35
1,2015-02-28,38294.70,26103.21
2,2015-03-31,34993.59,21325.19
3,2015-04-30,77728.49,15578.52
4,2015-05-31,18848.72,12708.71


In [57]:
# Reset index for chart axis labels
cons_df_area = cons_df_area.set_index(['Order Date']).resample('M').sum()
cons_df_area

,Profitable,Unprofitable
Order Date,,
2015-01-31,35650.12,21723.35
2015-02-28,38294.70,26103.21
2015-03-31,34993.59,21325.19
2015-04-30,77728.49,15578.52
2015-05-31,18848.72,12708.71
2015-06-30,83627.65,15196.12


In [95]:
# When adding stack=True, Y labels skew.  Fixed with NumeralTickFormatter
from bokeh.charts import Area, show
from bokeh.models import NumeralTickFormatter, HoverTool
title1 = df_segs.Segment.unique()[0]
# hover = HoverTool(
#         tooltips=[
#             ("Order Date", "$index"),
#             ("Profit", "$Profitable"),
#             ("desc", "@desc"),
#         ]
#     )
cons_area = Area(cons_df_area, title=title1, legend="top_left",
            xlabel='', ylabel='Profit', plot_width=700, plot_height=400,
            stack=True, 
            tools=[hover]
                )
cons_area.yaxis[0].formatter = NumeralTickFormatter(format="0,00")
show(cons_area)